# Лабораторная работа №3
## Выполнил студент группы БФИ2001 Чуйкин Максим

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [2]:
def prefix(s):
    P = [0]*len(s)
    j = 0
    i = 1
    
    while i < len(s):
        if s[j] != s[i]:
            if j > 0:
                j = P[j-1]
            else:          
                i += 1
        else:              
            P[i] = j + 1
            i += 1
            j += 1

    return P

def kmp(sub:str, s:str):
    k = 0
    l = 0
    P = prefix(sub)

    while k < len(s):
        if sub[l] == s[k]:
            k += 1
            l += 1

            if l == len(sub):
                return k - len(sub)

        elif l > 0:
            l = P[l-1]
        else:
            k += 1
    return -1

string = input("str \n")
sub = input("sub \n")
key = input("case sensitivity \n")

if key == '0':
	string = string.lower()
	sub = sub.lower()

print(kmp(sub, string))

str 
 Hellow world
sub 
 low
case sensitivity 
 1


3


#### Упрощенный алгоритм Бойера-Мура

In [7]:
def bm(str, key):
	if len(str) < len(key):
		return -1
	smesh = {}
	for i in range(len(key)):
		smesh[key[i]] = int(len(key) - i - 1)
	smesh[key[-1]] = len(key)

	i = j = k = len(key) - 1

	while j >= 0 and i < len(str):
		j = len(key) - 1
		k = i
		while j >= 0 and str[k] == key[j]:
			k -= 1
			j -= 1
		if smesh.get(str[k]) == None:
			i += len(key)
		else:
			i += smesh.get(str[k])

	if k >= len(str) - len(key):
		return -1
	else:
		return k + 1

def main():
	string = input("str \n")
	sub = input("sub \n")
	key = input("case sensitivity \n")

	if key == '0':
		string = string.lower()
		sub = sub.lower()
	result = bm(string, sub)
	print(result)

if __name__ == '__main__':
	main()

str 
 hellow world
sub 
 lowe
case sensitivity 
 0


-1


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [8]:
from heapq import heappop, heappush
from math import sqrt
import argparse

def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

    raise Exception("No solution?!")

def manh_dst_matrix(a, b, n):
    """Find manhattan distance between `a` and `b` in matrix of size `n`
    """
    return abs(a % n - b % n) + abs(a // n - b // n)

class chain15:
    def __str__(self):
        i = 0
        sstr = ""
        while i < self.size ** 2:
            sstr += str(self.board_state[i]) + " "
            if i % self.size == 3:
                sstr += "\n"
            i += 1
        return sstr

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state)
        self.size = int(sqrt(len(board_state)))
        self.history = history
        self.quad_size = int(self.size * self.size)

    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)
    
    def last_node(self):
        """Must be hashable value (list not hashable :( )
        """
        return str(self.board_state)
    
    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    def simple_heur(self):
        dst = 0
        for i in range(0, int(self.quad_size)):
            if (self.board_state[i] - 1) != i:
                dst += 1
        return dst

    def h(self):
        return self.manh_dst() + self.last_move()
    
    def g(self):
        return len(self.history)

    def f(self):
        return self.h() 

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # look at neighbours
        if zero_coord + 1 < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord + self.size < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        return neighs
    
start_state = [6, 12, 3, 4, 5, 1, 7, 10, 0, 8, 2, 11, 15, 14, 13, 9]
start = chain15(start_state)
end = chain15((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))

result = a_star(start, end.last_node())

for node in result.history:
    print(str(node))
    print("===========")
print(str(result))

6 12 3 4 
5 1 7 10 
0 8 2 11 
15 14 13 9 

6 12 3 4 
5 1 7 10 
8 0 2 11 
15 14 13 9 

6 12 3 4 
5 1 7 10 
8 2 0 11 
15 14 13 9 

6 12 3 4 
5 1 7 10 
8 2 11 0 
15 14 13 9 

6 12 3 4 
5 1 7 10 
8 2 11 9 
15 14 13 0 

6 12 3 4 
5 1 7 10 
8 2 11 9 
15 14 0 13 

6 12 3 4 
5 1 7 10 
8 2 0 9 
15 14 11 13 

6 12 3 4 
5 1 7 10 
8 2 9 0 
15 14 11 13 

6 12 3 4 
5 1 7 0 
8 2 9 10 
15 14 11 13 

6 12 3 4 
5 1 0 7 
8 2 9 10 
15 14 11 13 

6 12 3 4 
5 1 9 7 
8 2 0 10 
15 14 11 13 

6 12 3 4 
5 1 9 7 
8 2 11 10 
15 14 0 13 

6 12 3 4 
5 1 9 7 
8 2 11 10 
15 14 13 0 

6 12 3 4 
5 1 9 7 
8 2 11 0 
15 14 13 10 

6 12 3 4 
5 1 9 7 
8 2 0 11 
15 14 13 10 

6 12 3 4 
5 1 0 7 
8 2 9 11 
15 14 13 10 

6 12 0 4 
5 1 3 7 
8 2 9 11 
15 14 13 10 

6 0 12 4 
5 1 3 7 
8 2 9 11 
15 14 13 10 

6 1 12 4 
5 0 3 7 
8 2 9 11 
15 14 13 10 

6 1 12 4 
5 2 3 7 
8 0 9 11 
15 14 13 10 

6 1 12 4 
5 2 3 7 
8 9 0 11 
15 14 13 10 

6 1 12 4 
5 2 3 7 
8 9 11 0 
15 14 13 10 

6 1 12 4 
5 2 3 7 
8 9 11 10 
15 14 13 0 

6 1 12 4 
5

### Вывод